# Tutorial 1

## other doc sources
https://github.com/MarcToussaint/rai-maintenance/tree/master/help

In [1]:
import sys
sys.path.append('../ry')
from libry import *
from numpy import *

## Setting up a basic Config

The starting point is to create a `Configuration`.

In [2]:
K = Config()
D = K.view()

This shows an empty configuration. Tip: Make the view window appear "Always On Top" (right click on the window bar)

You can add things (objects, scene models, robots) to a configuration. As the view is strictly referring to the configuration, it automatically updates.

In [3]:
K.clear();
K.addFile('../rai-robotModels/pr2/pr2.g');
K.addFile('../test/kitchen.g');

Note that the view was updated automatically. (Internally, the view 'subscribes' to updates of the shared memory of Config).

In [4]:
K.addObject(name="ball", shape=ST.sphere, size=[.1], pos=[.8,.8,1.5], color=[1,1,0])

One can also add convex meshes (just passing the passing the vertex array), or use sphere-swept convex meshes (ssBox, capsule, sphere, etc)

In [5]:
K.addObject(name="hand", parent="pr2L", shape=ST.ssBox, size=[.2,.2,.1,.02], pos=[0,0,-.1], color=[1,1,0])

In this last example, the new object has another frame (pr2L) as *parent*. This means that it is permanently attached to this parent. pos and quat/rot are interpreted relative to the parent.

We can translate the framenames to attributes, so that they can be accessed by tab completion

In [6]:
class F(object):
    def __init__(self):
        print("bla")
    

for n in K.getFrameNames():
    setattr(F, n, n)

print(F.sink1)

sink1


## Joint and Frame State

A configuration is a tree of n frames. Every frame has a pose (position & quaternion), which is represented as a 7D vector (x,y,z, qw,qx,qy,qz). The frame state is the n-times-7 matrix, where the i-th row is the pose of the i-th frame.

A configuration also defines joints, which means that the relative transfromation from a parent to a child frame is parameterized by degrees-of-freedom (DOF). If the configuration has in total n DOFs, the joint state is a n-dimensional vector.

Setting the joint state implies computing all relative transformations, and then forward chaining all transformations to compute all frame poses. So setting the joint state also sets the frame state.
     
Setting the frame state allows you to set frame poses that are inconsistent/impossible w.r.t. the joints! Setting the frame state implies computing all relative transformations from the frame poses, and then assigning the joint state to the *projection( onto the actual DOFs

In [7]:
q = K.getJointState()
print('joint state: ', q)
print('joint names: ', K.getJointNames() )

joint state:  [ 0.    0.    0.    0.1   0.    0.   -1.    1.    0.4   0.5   0.5  -1.    1.
 -2.   -2.   -1.5   1.5  -0.5  -0.5  -0.5   0.5   0.1   0.1   0.01  0.01]
joint names:  ['worldTranslationRotation:0', 'worldTranslationRotation:1', 'worldTranslationRotation:2', 'torso_lift_joint', 'head_pan_joint', 'laser_tilt_mount_joint', 'r_shoulder_pan_joint', 'l_shoulder_pan_joint', 'head_tilt_joint', 'r_shoulder_lift_joint', 'l_shoulder_lift_joint', 'r_upper_arm_roll_joint', 'l_upper_arm_roll_joint', 'r_elbow_flex_joint', 'l_elbow_flex_joint', 'r_forearm_roll_joint', 'l_forearm_roll_joint', 'r_wrist_flex_joint', 'l_wrist_flex_joint', 'r_wrist_roll_joint', 'l_wrist_roll_joint', 'r_gripper_l_finger_joint', 'l_gripper_l_finger_joint', 'r_gripper_joint', 'l_gripper_joint']


Let's move the configuration by adding to the joint configuration

In [8]:
q[2] = q[2] + 1.
K.setJointState(q)

The *frame state* is a $n\times 7$ matrix, which contains for all of $n$ frames the 7D pose. A pose is stored as [p_x, p_y, p_z, q_w, q_x, q_y, q_z], with position p and quaternion q.

In [9]:
X0 = K.getFrameState()
print('frame state: ', X0)

frame state:  [[ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.47942554]
 [-0.02701518 -0.04207355  0.89067506 ...,  0.33900515 -0.62054477
   0.33900515]
 ..., 
 [ 0.5         2.          0.4        ...,  0.          0.          0.        ]
 [ 0.8         0.8         1.5        ...,  0.          0.          0.        ]
 [ 0.17550411  0.55772103  0.96712056 ...,  0.46891458  0.61993741
  -0.41697586]]


Let's do a questionable thing: adding .1 to all numbers in the pose matrix

In [10]:
X = X0 + .1
K.setFrameState(X)

The rows of X have non-normalized quaternions! These are normalized when setting the frame state.

Also, the frame poses are now *inconsistent* to the joint constraints! We can read out the projected joint state, set the joint state, and get a consistent state again:

In [11]:
K.setJointState( K.getJointState() )

Let's bring the configuration back into the state before the harsh *setFrame*

In [12]:
K.setFrameState(X0)

 ## Selecting joints

Often one would like to choose which joints are actually active, that is, which joints are referred to in q. This allows one to sub-selection joints and work only with projections of the full configuration state. This changes the joint state dimensionality, including ordering of entries in q.

However, the frame state is invariant against such selection of active joints.

In [13]:
K.selectJointsByTag(["armL","base"])
q = K.getJointState()
print('joint state: ', q)
print('joint names: ', K.getJointNames() )

joint state:  [ 0.       0.       1.       1.00001  0.50001  1.00001 -2.00001  1.50001
 -0.50001  0.50001]
joint names:  ['worldTranslationRotation:0', 'worldTranslationRotation:1', 'worldTranslationRotation:2', 'l_shoulder_pan_joint', 'l_shoulder_lift_joint', 'l_upper_arm_roll_joint', 'l_elbow_flex_joint', 'l_forearm_roll_joint', 'l_wrist_flex_joint', 'l_wrist_roll_joint']


## Features & Jacobians

A core part of rai defines features over configurations. A feature is a differentiable mapping from a configuration (or set of configurations) to a vector. Starndard features are "position-of-endeffector-X" or "distance/penetration-between-convex-shapes-A-and-B", etc. But there are many, many more features defined in rai, like error of Newton-Euler-equations for an object, total energy of the system, etc. Defining differentiable is the core of many functionalities in the rai code.

Let's define a basic feature over C: the 3D (world coordinate) position of pr2L (left hand)

In [14]:
F = K.feature(FS.position, ["pr2L"])

We can now evaluate the feature, and also get the Jacobian:

In [15]:
print(F.description(K))

[y,J] = F.eval(K)
print('hand position =', y)
print('Jacobian =', J)

Default-0-pos-pr2L-WORLD
hand position = [ 0.07798888  0.55020201  0.9462799 ]
Jacobian = [[ 1.          0.         -0.68507935 -0.48015475  0.1009033   0.12598409
  -0.00526037 -0.11617989 -0.03910962 -0.0010683 ]
 [ 0.          1.         -0.10767784  0.24947488 -0.05494612 -0.04034564
   0.49557667  0.23261832 -0.04015311  0.10840063]
 [ 0.          0.          0.11906956  0.07121761 -0.15810018 -0.40055836
  -0.15183121  0.13599561  0.19748679 -0.03411141]]


Another example

In [16]:
F2 = K.feature(FS.distance, ["hand", "ball"])
print(F2.description(K))

PairCollision-hand-ball


In [17]:
F2.eval(K)

(array([-0.65768159]),
 array([[  7.96770252e-01,   2.30277394e-01,  -5.39602246e-01,
          -3.13678306e-01,  -3.90917875e-02,  -9.45176564e-02,
           1.34070815e-03,   9.11744758e-03,   3.91519364e-02,
          -3.87493737e-05]]))

When you call a feature on a *tuple* of configurations, by default it computes the difference, acceleration, etc, w.r.t. these configurations

In [18]:
C2 = Config()
C2.copy(K)  #this replicates the whole structure
V2 = C2.view()

In [19]:
F.eval((K,C2))[0]

array([ 0.,  0.,  0.])

This should be zero. To see a difference, let's move the 2nd configuration:

In [20]:
# just to see a difference between the two:
q = C2.getJointState()
q = q - .1
C2.setJointState(q)
y = F.eval((K,C2))[0]
print('hand difference (y(C2) - y(K)) =', y)

hand difference (y(C2) - y(K)) = [-0.02791135 -0.20312554  0.04771979]


An acceleration example:

In [21]:
C3 = Config()
C3.copy(K);
C3.setJointState(q + .2);

In [22]:
(y,J) = F.eval((K, C2, C3))
print('hand acceleration = ', y)
print('shape of Jacobian =', J.shape)

hand acceleration =  [ 0.05117693  0.58251037 -0.16073462]
shape of Jacobian = (3, 30)


Note that the Jacobian is now w.r.t. all three configurations! It is of size 3x3xdim(q). Let's retrieve the Jacobian w.r.t. C3 only:

In [23]:
J = J.reshape((3,3,q.size))
print('shape of Jacobian =', J.shape)
J[:,1,:]

shape of Jacobian = (3, 3, 10)


array([[ -2.00000000e+00,  -0.00000000e+00,   8.94152952e-01,
          7.38760369e-01,   4.69531268e-02,  -3.97202637e-01,
         -2.05993379e-01,  -7.03166221e-02,   5.14244803e-02,
          1.07705377e-07],
       [ -0.00000000e+00,  -2.00000000e+00,  -3.00155050e-01,
         -6.56847293e-01,  -2.01244293e-01,   2.30244444e-01,
         -9.77092581e-01,  -2.29438647e-01,  -9.08138357e-02,
          3.19001014e-07],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.23028316e-16,   3.70194756e-01,   7.43774595e-01,
          1.84245211e-02,   4.11591878e-02,  -4.18380024e-01,
         -1.11742325e-07]])

Another example, when the dimensions of K and C2 are different:

In [24]:
C2.selectJointsByTag(['armL'])
(y,J) = F.eval((K,C2))
print('shape of Jacobian =', J.shape)
print('dimensions of configurations =', (K.getJointDimension(), C2.getJointDimension()))

shape of Jacobian = (3, 17)
dimensions of configurations = (10, 7)


Finally, we can linearly transform features by setting 'scale' and 'target':

In [25]:
#F.scale = 10.
#F.target = [0., 0., 1.];
#  y = F(C);
#  //.. now y = F.scale * (f(C) - F.target), which means y is zero if
#  //the feature f(C) is equal to the target (here, if the hand is in world
#  //position (0,0,1) )
#
#  //F.scale can also be a matrix, which can transform the feature also to lower dimensionality
#  F.scale = arr(1,3,{0., 0., 1.}); //defines the 1-times-3 matrix (0 0 1)
#  y = F(C);
#  //.. now y is 1-dimensional and captures only the z-position 

# THE REST IS PRELIM

We can also add a frame, attached to the head, which has no shape associated to it, but create a view is associated with that frame:

In [26]:
K.addFrame(name='camera', parent='head_tilt_link', args='Q:<d(-90 1 0 0) d(180 0 0 1)> focalLength:.3')
C = K.view(frame='camera')

When we move the robot, that view moves with it:

In [27]:
K.setJointState(q=asarray([1.]), joints=['head_pan_joint'])

To close a view (or destroy a handle to a computational module), we reassign it to zero. We can also remove a frame from the configuration.

In [28]:
C = 0
K.delFrame('camera')

This solves a simple IK problem, defined by an equality constraint on the difference in position of 'ball' and 'hand'

In [29]:
X0 = K.getFrameState()
IK = K.komo_IK()
IK.addObjective(type=OT.eq, feature=FS.positionDiff, frames=['ball', 'hand'])
IK.optimize()
K.setFrameState( IK.getConfiguration(0) )

In [30]:
IK.getReport()

[{'feature': 'Transition',
  'scale': 0.1,
  'sos_value': 16.181251386054,
  'type': 'sos'},
 {'description': 'QuaternionNorms',
  'scale': 3.0,
  'sos_value': 0.0,
  'type': 'sos'},
 {'eq_sumOfAbs': 3.689615277657765e-05,
  'feature': 'posDiff',
  'o1': 'ball',
  'o2': 'hand',
  'scale': 10.0,
  'type': 'eq'}]

We can reuse the optimization object, change the objective a bit (now the position difference is constrained to be [.1,.1,.1] in world coordinates), and reoptimize

In [31]:
IK.clearObjectives()
IK.addObjective(type=OT.eq, feature=FS.positionDiff, frames=['hand', 'ball'], target=[.1, .1, .1])
IK.optimize()
K.setFrameState( IK.getConfiguration(0) )

TODO demos:

* rename Camera -> View
* copy configurations
* have multiple configurations and views in parallel
* selecting/modifying DOFs (i.e., which joints are considered DOFs)
* I/O with other file formats?

In [32]:
K.setFrameState(X0)

In [33]:
K=0

In [34]:
D=0